# global TOMAWAC simulations 

We'll generate a world contiguous mesh for global wave simulations.

### install base environment 
To be able to run this notebook, first install the pre-requisites for seamsh: 

```bash
mamba create -n tomawac_mesh gdal gmsh scipy python=3.12
```

#### Optionally 
create then a virtual environment on top of conda: 

```bash
python -mvenv .venv 
source .venv/bin/activate
```

### install libraries 
```bash
pip install -e .
```

You're ready to run the notebook, and generate different version of the mesh.

This notebook contains to following sections: 

1. gather data: 
   * A. GEBCO bathy through [`seareport_data`](https://github.com/seareport/seareport_data)
   * B. coastlines from natural earth coasltines
2. Prepare the distance fields from the above data
   * only coastline is used to constrain with the Distance from the shore, but you can activate more contraints if necessary
3. Mesh
4. Interpolate bathy onto the mesh
5. Convert to Selafin (using `xarray-selafin` available via `pip`)
6. Generate the CLI file
7. Interpolate ERA5 wind data onto the mesh
8. Convert to wind Selafin binary
9. Visualise TOMAWAC results directly in the notebook: 
![WAC_global](assets/max_Hm0_TOMAWAC.png)

## 1 - Generate the mesh

### Particularities of the meshes available in the `mesh/` folder: 
 1. For TOMAWAC simulations, I needed to add a hole in the pole. Only because of the triangle(s) that contain the pole (bug in routine `GEOELT.f`).
 2. The mesh is contiguous, e.g. it wrap around the globe and the triangles at the dateline (+/- 180°) are connected alltogether. 
 3. The only constraint for the mesh size is the distance from the coast. You can activate the wavelength contrainst (proportional to `sqrt(depth)`) or the bathy gradient constraint.

## 2 - Run the model

You will need to compile and run TELEMAC from the `main` branch, because it contains the latest version of 1D time series exports, very useful for global model analysis (and comparison against buoys).

All the modified routines to run TOMAWAC global are in `wac/princi/` folder.

Look for `SEB` or `TOM` for the modified parts of the source code.


## Get the whole project: 

available on Google drive : 
https://drive.google.com/drive/folders/1SMNkHymAeuRfKh27pwPf7TgdIlOhTBvI?usp=sharing

In [ ]:
import seamsh
import numpy as np
import xarray as xr
import seareport_data as D
import utils
import hvplot.pandas
import hvplot.xarray

## Mesh settings 

here are the main parameters used for the mesher settings: 

In [ ]:
# Meshing parameters for contributions
OPTS = {
    "max": 150000,      # max resolution
    "min": 10000,        # min resolution
    "dist": 0.15,       # The rate of expansion in decimal percent from the shoreline
    "wl": 50,           # (not used here) number of element to resolve WL
    "m2": 12.42 * 3600, # (not used here) for wavelength: M2 period in seconds
    "g": 9.81,          # (not used here) for wavelength: m/s^2
    "slope": 5,         # (not used here) number of element to resolve bathy gradient
    "grade": 1.5,       # the rate of growth in decimal percent
}
REMOVE_POLE = True      # for tomawac simulations

## Coastlines 

In [ ]:
SHP_IN = "ne_10m_coastline/ne_10m_coastline.shp"
SHPOUT = "ne_10m_coastline/ne_10m_coastline_poly.shp"
POLE= "ne_10m_coastline/pole_ring.shp"
utils.get_ne_coastline(SHP_IN)
utils.remove_small_islands(SHP_IN, SHPOUT, OPTS['min'])
utils.add_hole_in_pole(POLE)

### define the CRS for the meshing 

In [ ]:
from osgeo import osr
domain_srs = osr.SpatialReference()
domain_srs.ImportFromProj4("+ellps=WGS84 +proj=stere +lat_0=90")
# "Cartesian" projection (i.e. no projection) is used to compute the distance to the coast.
cart_srs = osr.SpatialReference()
cart_srs.ImportFromProj4("+ellps=WGS84 +proj=cart +units=m +x_0=0 +y_0=0")
wgs84_srs = osr.SpatialReference()
wgs84_srs.ImportFromProj4("+ellps=WGS84 +proj=longlat +datum=WGS84 +no_defs")

## Distance field 

In [ ]:
domain = seamsh.geometry.Domain(domain_srs)
domain.add_boundary_curves_shp(SHPOUT, "featurecla", seamsh.geometry.CurveType.POLYLINE)
dist_coast = seamsh.field.Distance(domain, OPTS["min"]/2, projection=cart_srs)

## Bathy field (not used here)

In [ ]:
bathy = D.gebco_ds("sub_ice")
bathy_subset = bathy.isel(lon=slice(0,-1,10), lat=slice(0,-1,10))
bathy_grad = utils.calc_gradient(bathy_subset, 'elevation')
bathy_grad

In [ ]:
! mkdir -p data

In [ ]:
BATHY = "data/gebco_2024_1000_4k.tif"
BATHY_GRADIENT = "data/gebco_2024_grad_smooth_1000_4k.tif"
utils.to_raster(bathy_subset.elevation, BATHY)
utils.to_raster(bathy_grad.gradient, BATHY_GRADIENT)

In [ ]:
bath_field = seamsh.field.Raster(BATHY)
bath_field._projection = osr.SpatialReference()
bath_field._projection.ImportFromProj4("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
grad_field = seamsh.field.Raster(BATHY_GRADIENT)
grad_field._projection = osr.SpatialReference()
grad_field._projection.ImportFromProj4("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
# %%

## MESH

### Mesh size function

In [ ]:
# it is necessary to convert the mesh size to stereographic coordinates
def mesh_size(x, projection,R = 6371000):
    depth = bath_field(x, projection)
    depth[depth>0] = 0
    grad = grad_field(x, projection)
    lon, lat = utils.stereo_to_wgs84(x, source_epsg=projection)
    s_wl = OPTS["m2"] / OPTS["wl"] * np.sqrt(9.81 * - depth)
    s_wl = np.clip(s_wl, OPTS["min"]*2,None)
    s_grad = abs(depth) / (grad + 1e-10)*(2*np.pi / OPTS["slope"])
    s_grad = np.clip(s_grad, OPTS["min"]*3,None)
    s_coast = dist_coast(x, projection)* OPTS["grade"] + OPTS["min"]

    # s_final = np.c_[s_coast, s_wl, s_grad].min(axis=1)
    s_final = s_coast

    stereo_factor = 2/(1+x[:,0]**2/R**2+x[:,1]**2/R**2) * (3/4*np.cos(np.deg2rad(lat+90))+5/4)
    return np.clip(s_final,OPTS["min"],OPTS["max"])/stereo_factor

Coarsen boundaries

In [ ]:
coarse = seamsh.geometry.coarsen_boundaries(domain, (0, 0), domain_srs, mesh_size)
if REMOVE_POLE:
    coarse.add_boundary_curves_shp(POLE,"featurecla", seamsh.geometry.CurveType.POLYLINE)
else: 
    x = [[0., 0.]]
    coarse.add_interior_points(x,"featurecla",domain_srs)

finally mesh 

In [ ]:
seamsh.gmsh.mesh(
    coarse, 
    "natural_earth.msh", 
    mesh_size, 
    output_srs=domain_srs,
    intermediate_file_name="-",
    # binary=True
)
seamsh.gmsh.reproject("natural_earth.msh", domain_srs, "natural_earth_wgs84.msh", wgs84_srs)

## Interpolate Bathy

In [ ]:
import gmsh
gmsh.open("natural_earth_wgs84.msh")
tri_i, tri_n = gmsh.model.mesh.getElementsByType(2)
tri_n = tri_n.reshape([-1, 3])
node_i, nodes, _ = gmsh.model.mesh.getNodes()
nodes = nodes.reshape([-1, 3])
x = nodes[:, 0]
y = nodes[:, 1]
z = nodes[:, 2]
element = np.subtract(tri_n, 1)

In [ ]:
subset = bathy.isel(lon=slice(0, -1, 10), lat=slice(0, -1, 10)).pad(lon=(1,1),lat=(1,1),mode="edge")
lon_values = subset.lon.values
lat_values = subset.lat.values
lon_values[-1] = 180.0
lon_values[0] = -180.0
lat_values[-1] = 90.0
lat_values[0] = -90.0
subset = subset.assign_coords(lon=lon_values,lat=lat_values)
subset

In [ ]:
from scipy.interpolate import RegularGridInterpolator as RGI
RG = RGI((subset.lon, subset.lat), subset.elevation.data.T, method="linear")

## export to Selafin

In [ ]:
!mkdir -p mesh

In [ ]:
from xarray_selafin.xarray_backend import SelafinAccessor
import pandas as pd

slf_ds = xr.Dataset(
    coords={
        "time": [pd.Timestamp.now()],
        "x": ("node", x),
        "y": ("node", y),
    },
    data_vars={
        "B" : (("time", "node"), [RG((x,y))])
    }
)
slf_ds.attrs['ikle2'] = element + 1
filebase = f"mesh/global_{OPTS["min"]/1000}-{int(OPTS["max"]/1000)}km-smo{OPTS["grade"]-1}-{len(slf_ds.x)}nodes"
slf_ds.selafin.write(filebase+".slf")
# export CLI file
_=utils.export_cli(slf_ds, filebase+".cli")

In [ ]:
slf_ds

In [ ]:
filebase

In [ ]:
slf_ds.B.hvplot.scatter(
    x="x",
    y="y",
    c='B',
    cmap="fire",
    s=1,
    clim=(-5000, 0),
    ).opts(
        width=800,
        height=500,
    )

## Interpolate wind

In [ ]:
atm_nc = xr.open_dataset("data/era5_202307_uvp.nc")
atm_nc

In [ ]:
lon_values = atm_nc.longitude.values
lat_values = atm_nc.latitude.values
lon_values[-1] = 360.0
atm_nc = atm_nc.assign_coords(lon=lon_values)

In [ ]:
atm_data = dict()

for var in ["msl", "u10", "v10"]: 
    RG = RGI((atm_nc.longitude, atm_nc.latitude), atm_nc[var].data.T, method="linear")
    atm_data[var] = RG((slf_ds.x.data % 360, slf_ds.y.data))

slf_wind =xr.Dataset(coords = {
    "x": ("node", slf_ds.x.data),
    "y": ("node", slf_ds.y.data),
    "time": atm_nc.time.data
    },
    data_vars={
        "PATM": (("node", "time"), atm_data["msl"]),
        "WINDX": (("node", "time"), atm_data["u10"]),
        "WINDY": (("node", "time"), atm_data["v10"]),
    }
)
slf_wind.attrs["ikle2"] = slf_ds.ikle2
slf_wind.attrs["variables"] = {'PATM': ('PATM', 'PASCAL'), 'WINDX': ('WINDX', 'M/S'), 'WINDY': ('WINDY', 'M/S')}

In [ ]:
slf_wind.selafin.write(filebase+"_wind.slf")

### Inspect results

In [ ]:
res2d = xr.open_dataset("wac/r2d_global_tom.slf")
res1d = xr.open_dataset("wac/r1d_global_tom.slf")

In [ ]:
res2d.max(dim="time").WH.hvplot.scatter(x="x",
    y="y",
    c='WH',
    cmap="rainbow4",
    s=1,   
).opts(
    clim=(0, 10),
    width=800,
    height=500,
    title="Max Hm0 over July 2023"
)

In [ ]:
res1d.max(dim="time").WH.hvplot.scatter(x="x",
    y="y",
    c='WH',
    cmap="rainbow4",
    hover_cols=["node"]
).opts(
    clim=(0, 10),
    width=800,
    height=500,
    title="Max Hm0 over July 2023"
)

In [ ]:
res1d.isel(node=389).WH.hvplot(
).opts(
    width=800,
    height=500,
    title=f"Hm0 over July 2023, for station located at {res1d.isel(node=389).x.values:0.2f}°E, {res1d.isel(node=389).y.values:0.2f}°N"
)